In [39]:
import pandas as pd
import os
import numpy as np

In [40]:
data1 = pd.read_csv('datasets_64642_127042_BankNote.csv')
data1.head()


,variance,skewness,kurtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [41]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   variance   1372 non-null   float64
 1   skewness   1372 non-null   float64
 2   kurtosis   1372 non-null   float64
 3   entropy    1372 non-null   float64
 4   class      1372 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.7 KB


In [42]:
data1["class"].value_counts()

0    762
1    610
Name: class, dtype: int64

In [43]:
# Stratified Sampling using Scikit-learn's Stratified Shuffle Split Class
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in split.split(data1, data1["class"]):
    strat_train_set = data1.loc[train_index]
    strat_test_set = data1.loc[test_index]

In [44]:
 strat_test_set["class"].value_counts()

0    191
1    152
Name: class, dtype: int64

In [45]:
strat_train_set["class"].value_counts()

0    571
1    458
Name: class, dtype: int64

In [46]:
train_set = strat_train_set.drop("class", axis=1) # drop labels for training set
train_labels = strat_train_set["class"].copy()
test_set = strat_test_set.drop("class", axis=1) # drop labels for testing set
test_labels = strat_test_set["class"].copy()

In [47]:
train_labels

603     0
880     1
107     0
286     0
869     1
       ..
437     0
45      0
1068    1
1080    1
59      0
Name: class, Length: 1029, dtype: int64

In [48]:
test_labels

454     0
867     1
64      0
927     1
1186    1
       ..
992     1
128     0
828     1
32      0
1328    1
Name: class, Length: 343, dtype: int64

In [49]:
!pip install fuzzy-c-means
from fcmeans import FCM
tr_set=train_set.T
fcm = FCM(n_clusters=4,random_state=42)
fcm.fit(tr_set.to_numpy())
tr_fin  = fcm.centers.T

In [50]:
te_set=test_set.T
fcm = FCM(n_clusters=4, random_state=42)
fcm.fit(te_set.to_numpy())
te_fin  = fcm.centers.T

In [51]:
tr_fin.shape

(1029, 4)

In [52]:
train_labels.shape

(1029,)

In [53]:
from sklearn.metrics import matthews_corrcoef
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameters = {"kernel": ["linear"],
              "class_weight": ["balanced"],
              "tol": [1e-3],
              "random_state": [42]
             }
clf2 = SVC()

clf = GridSearchCV(clf2, parameters, scoring = 'balanced_accuracy', cv=10)
res1=clf.fit(train_set, train_labels)
res1.best_estimator_

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [54]:
res1.best_params_

{'class_weight': 'balanced',
 'kernel': 'linear',
 'random_state': 42,
 'tol': 0.001}

In [55]:
res1.best_score_

0.994542202582919

In [56]:
y_test_pred=res1.best_estimator_.predict(te_fin)


In [57]:
import sklearn
from sklearn.metrics import accuracy_score
sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100

88.92128279883383

In [58]:
#error rate=100-accuracy
100-(sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100)

11.078717201166171

In [59]:
matthews_corrcoef(test_labels, y_test_pred)*100

79.01501073413526

In [60]:
from sklearn.metrics import precision_score
sklearn.metrics.precision_score(test_labels, y_test_pred, pos_label= 1)*100

81.66666666666667

In [61]:
from sklearn.metrics import recall_score
sklearn.metrics.recall_score(test_labels, y_test_pred, pos_label= 1)*100

96.71052631578947

In [62]:
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(test_labels, y_test_pred,  pos_label= 1)*100

88.55421686746988